In [1]:
import pstats

## Código original

Todo en Python

In [2]:
p = pstats.Stats('../../output/profiles/prop')
p.sort_stats('cumulative').print_stats(20)

Mon Jun 13 16:15:04 2016    ../../output/profiles/prop

         21748402 function calls (21746006 primitive calls) in 351.969 seconds

   Ordered by: cumulative time
   List reduced from 881 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    128/1    0.008    0.000  351.970  351.970 {built-in method builtins.exec}
        1    0.001    0.001  351.970  351.970 test_time.py:1(<module>)
       28    0.177    0.006  351.706   12.561 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    33500   27.871    0.001  347.449    0.010 /home/fnbellomo/Desktop/ten/ten/mechanisms.py:27(forster)
  4191930  124.165    0.000  166.385    0.000 /home/fnbellomo/Desktop/ten/ten/mechanisms.py:162(__transfer_rate)
  2062465   31.399    0.000  152.377    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
  2246171   90.432    0.000  112.874    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:15(generate_random_points_in_sphere)
  6371073

Podemos observar que la función `__transfer_rate` (que calcula la taza de transferencia del exiton a cualquier aceptor mediante un mecanismo de forster) tarda en total 124 seg.
Ahora, la misma simulación, pero con esta función implementada en Fortran90

In [3]:
p = pstats.Stats('../../output/profiles/prop2')
p.sort_stats('cumulative').print_stats(20)

Tue Jun 14 09:37:11 2016    ../../output/profiles/prop2

         13779389 function calls (13776987 primitive calls) in 203.436 seconds

   Ordered by: cumulative time
   List reduced from 896 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    129/1    0.008    0.000  203.436  203.436 {built-in method builtins.exec}
        1    0.001    0.001  203.436  203.436 test_time.py:1(<module>)
       28    0.193    0.007  203.163    7.256 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    34000   39.293    0.001  198.823    0.006 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2128783   32.715    0.000  158.611    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
  2315887   93.238    0.000  115.835    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:15(generate_random_points_in_sphere)
  4512670   14.714    0.000   14.714    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
  22478

Ahora no aparece. Si comparamos los tiempos totales, en el primer caso es de 352 segundos mientras que en el segundo caso es de 203 segundos. La diferencia entre estos tiempos es la función `__transfer_rate`

In [4]:
p = pstats.Stats('../../output/profiles/prop3')
p.sort_stats('cumulative').print_stats(20)

Wed Jun 15 08:57:55 2016    ../../output/profiles/prop3

         10237062 function calls (10234654 primitive calls) in 93.374 seconds

   Ordered by: cumulative time
   List reduced from 896 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    129/1    0.007    0.000   93.375   93.375 {built-in method builtins.exec}
        1    0.001    0.001   93.375   93.375 test_time.py:1(<module>)
       28    0.136    0.005   93.129    3.326 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    35500   33.268    0.001   92.262    0.003 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2333560   34.981    0.000   58.027    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:124(walk)
  2576240   13.651    0.000   13.651    0.000 {built-in method builtins.sum}
  2647268    9.690    0.000    9.724    0.000 /home/fnbellomo/Desktop/ten/ten/utils.py:17(generate_random_points_in_sphere)
  2404560    0.967    0.000    0.9

In [5]:
p = pstats.Stats('../../output/profiles/prop4')
p.sort_stats('cumulative').print_stats(20)

Wed Jun 15 09:22:16 2016    ../../output/profiles/prop4

         4751282 function calls (4748875 primitive calls) in 36.753 seconds

   Ordered by: cumulative time
   List reduced from 880 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    129/1    0.008    0.000   36.754   36.754 {built-in method builtins.exec}
        1    0.001    0.001   36.754   36.754 test_time.py:1(<module>)
       28    0.121    0.004   36.509    1.304 /home/fnbellomo/Desktop/ten/ten/experiments.py:13(quenching)
    33500   28.142    0.001   35.702    0.001 /home/fnbellomo/Desktop/ten/ten/mechanisms/mechanisms.py:29(forster)
  2176426    6.793    0.000    6.793    0.000 /home/fnbellomo/Desktop/ten/ten/exciter.py:125(walk)
  2243426    0.767    0.000    0.767    0.000 {method 'random_sample' of 'mtrand.RandomState' objects}
    33500    0.089    0.000    0.364    0.000 /home/fnbellomo/Desktop/ten/ten/nanoparticle.py:207(excite)
    33500    0.050    0.000 